In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

from statsmodels.regression.quantile_regression import QuantReg

pd.set_option('max_rows', 10)



# Financial Sector - Estimation

All data series were downloaded from the [World Bank Website](http://data.worldbank.org)

* Financial Depth: Domestic credit provided by financial sector (% of GDP) - EXPLANATORY VARIABLE
* Real GDP Per Capita: GDP per capita, PPP (current international $) - RESPONSE VARIABLE
* Inflation Rate Inflation, consumer prices (annual %) - CONTROL VARIABLE
* Merchandise Trade: Merchandise trade (% of GDP) - CONTROL VARIABLE
* Primary completion rate, total (% of relevant age group) - We opted for this as a education proxy - CONTROL VARIABLE


## Load

In [2]:
df = pd.read_excel('Data.xlsx',None)

In [3]:
def sortDf(df,key):
    df.drop(['Country Name','Indicator Code','Indicator Name',], axis=1, inplace=True)
    df = pd.DataFrame(df.set_index('Country Code').stack())
    df.columns = [key]
    return df

In [4]:
dfsList = []
for key in df.keys():
    dfsList.append(sortDf(df[key],key))

In [5]:
df = pd.concat(dfsList,axis=1)
df

InflationRate         Pop  FinancDepth  MerchanTrade  \
Country Code                                                              
ABW          1960            NaN     54208.0          NaN           NaN   
             1961            NaN     55435.0          NaN           NaN   
             1962            NaN     56226.0          NaN           NaN   
             1963            NaN     56697.0          NaN           NaN   
             1964            NaN     57029.0          NaN           NaN   
...                          ...         ...          ...           ...   
ZWE          2011       3.277509  14255592.0          NaN     72.215785   
             2012       3.922235  14565482.0          NaN     66.833044   
             2013       1.631622  14898092.0          NaN     57.873718   
             2014      -0.217286  15245855.0          NaN     51.164230   
             2015      -2.398710  15602751.0          NaN     48.338968   

                   GrossCapital  RealGdpPerCapita  PrimaryRateCompletion  
Country Code                                                              
ABW          1960           NaN               NaN                    NaN  
             1961           NaN               NaN                    NaN  
             1962           NaN               NaN                    NaN  
             1963           NaN               NaN                    NaN  
             1964           NaN               NaN                    NaN  
...                         ...               ...                    ...  
ZWE          2011     22.392920       1523.621784                    NaN  
             2012     13.612891       1679.125594              89.225418  
             2013     13.033016       1743.210783              90.249062  
             2014     13.236799       1798.059812                    NaN  
             2015     13.239581       1793.586092                    NaN  

[14628 rows x 7 columns]

# Selecting Our Sample

We will try to use a similar time window of 15 years, thus from 2000 to 2015

In [6]:
df = df.swaplevel()

listOfYears = []
for i in range(1999,2016):
    listOfYears.append(str(i))

df.sort_index(inplace=True)
idx = pd.IndexSlice
df = df.loc[idx[listOfYears,:],:]
df

InflationRate         Pop  FinancDepth  MerchanTrade  \
     Country Code                                                         
1999 ABW                2.280372     89004.0    44.554334    196.364238   
     AFG                     NaN  19038420.0          NaN           NaN   
     AGO              248.195902  14601983.0     7.219949    134.342357   
     ALB                0.389438   3108778.0    50.112177     44.085370   
     AND                     NaN     64161.0          NaN           NaN   
...                          ...         ...          ...           ...   
2015 WSM                0.724470    193228.0    76.109221     50.813106   
     YEM                     NaN  26832215.0          NaN           NaN   
     ZAF                4.588271  54956920.0   180.050114     59.556919   
     ZMB               10.100695  16211767.0    29.448022     72.693816   
     ZWE               -2.398710  15602751.0          NaN     48.338968   

                   GrossCapital  RealGdpPerCapita  PrimaryRateCompletion  
     Country Code                                                         
1999 ABW              27.723498               NaN             101.619118  
     AFG                    NaN               NaN                    NaN  
     AGO              28.899574       2972.321938                    NaN  
     ALB              22.986393       3684.945153                    NaN  
     AND                    NaN               NaN                    NaN  
...                         ...               ...                    ...  
2015 WSM                    NaN       5922.815359                    NaN  
     YEM                    NaN               NaN                    NaN  
     ZAF              19.508398      13165.148024                    NaN  
     ZMB                    NaN       3852.658974                    NaN  
     ZWE              13.239581       1793.586092                    NaN  

[4469 rows x 7 columns]

Now we have to find countries with complete data

In [7]:
df = df.swaplevel().sort_index()

In [8]:
pd.set_option('max_rows', 500)
~(df.isnull().groupby(level="Country Code").any().any(axis=1))

Country Code
ABW    False
AFG    False
AGO    False
ALB    False
AND    False
ARB    False
ARE    False
ARG    False
ARM    False
ASM    False
ATG    False
AUS    False
AUT    False
AZE    False
BDI    False
BEL    False
BEN    False
BFA    False
BGD    False
BGR    False
BHR    False
BHS    False
BIH    False
BLR    False
BLZ    False
BMU    False
BOL    False
BRA    False
BRB    False
BRN    False
BTN    False
BWA    False
CAF    False
CAN    False
CEB    False
CHE    False
CHI    False
CHL    False
CHN    False
CIV    False
CMR    False
COD    False
COG    False
COL    False
COM    False
CPV    False
CRI    False
CSS    False
CUB    False
CUW    False
CYM    False
CYP    False
CZE    False
DEU    False
DJI    False
DMA    False
DNK    False
DOM    False
DZA    False
EAP    False
EAR    False
EAS    False
ECA    False
ECS    False
ECU    False
EGY    False
EMU    False
ERI    False
ESP    False
EST    False
ETH    False
EUU    False
FCS    False
FIN    False
FJI    False
FRA    False

In [9]:
df.loc["USA"]

,InflationRate,Pop,FinancDepth,MerchanTrade,GrossCapital,RealGdpPerCapita,PrimaryRateCompletion
1999,2.188027,279040000.0,201.263956,18.168982,23.317953,34620.928899,NaN
2000,3.376857,282162411.0,190.962275,19.846980,23.568925,36449.855116,NaN
2001,2.826171,284968955.0,198.587775,17.965653,22.051491,37273.618103,NaN
2002,1.586032,287625193.0,191.785229,17.247375,21.576522,38166.037841,NaN
2003,2.270095,290107933.0,206.579807,17.616881,21.659967,39677.198348,NaN
2004,2.677237,292805298.0,212.918016,19.067770,22.526727,41921.809762,NaN
2005,3.392747,295516599.0,216.329179,20.114886,23.222953,44307.920585,NaN
2006,3.225944,298379912.0,226.324174,21.247602,23.333015,46437.067117,NaN
2007,2.852672,301231207.0,235.894430,21.886185,22.351353,48061.537661,NaN
2008,3.839100,304093966.0,216.762891,23.486835,20.786244,48401.427340,NaN


It appears that Primary Rate completion is not filled for many countries. Thus we will drop it for now.

In [10]:
df.drop('PrimaryRateCompletion',axis=1,inplace=True)

In [11]:
sampleCountries = ~(df.isnull().groupby(level="Country Code").any().any(axis=1))

In [12]:
sampleCountries = sampleCountries[sampleCountries==True].index
len(sampleCountries)

101

We have 101 countries in the sample. Which is more than the original paper with 77. We have our samples.

In [13]:
df = df.loc[list(sampleCountries)]

# Data Manipulations

## Calculate GDP Growth Rate

Since we only have the absolute value of the variable

In [14]:
df['GdpRate'] = df['RealGdpPerCapita'].groupby(level="Country Code").shift(0) / df['RealGdpPerCapita'].groupby(level="Country Code").shift(1)

## Ln() of Inflation

In [15]:
df['LnInflation'] = np.log(df['InflationRate'] /  100)

In [16]:
df.head()

InflationRate         Pop  FinancDepth  MerchanTrade  \
Country Code                                                              
AGO          1999     248.195902  14601983.0     7.219949    134.342357   
             2000     324.996872  15058638.0   -14.755435    120.059565   
             2001     152.561022  15562791.0    -0.319143    108.694391   
             2002     108.897436  16109696.0     5.004125     96.724531   
             2003      98.223718  16691395.0     6.626453    105.631501   

                   GrossCapital  RealGdpPerCapita   GdpRate  LnInflation  
Country Code                                                              
AGO          1999     28.899574       2972.321938       NaN     0.909048  
             2000     12.755027       3036.559728  1.021612     1.178645  
             2001     13.450919       3131.995858  1.031429     0.422394  
             2002     37.296420       2858.897069  0.912804     0.085236  
             2003     38.228106       2961.964572  1.036051    -0.017922

# Estimation

In [17]:
finalSample = df.groupby(level='Country Code').mean()
finalSample.head()

,InflationRate,Pop,FinancDepth,MerchanTrade,GrossCapital,RealGdpPerCapita,GdpRate,LnInflation
Country Code,,,,,,,,
AGO,65.629633,1.940390e+07,10.225204,94.360654,21.209882,5076.298701,1.061294,-1.261715
ALB,2.463691,2.977762e+06,58.261895,47.337332,33.164009,7593.348105,1.073425,-4.065030
ARM,3.937327,3.012460e+06,21.674713,54.208403,26.849415,5432.326964,1.092614,-3.416228
ATG,1.949115,8.438524e+04,85.296767,53.736358,28.899016,19611.929659,1.031085,-4.005236
AUS,2.794667,2.114647e+07,131.351110,33.914163,26.982632,35904.048442,1.037834,-3.633616


## Q=0.1

In [18]:
model = QuantReg(finalSample['GdpRate'], finalSample[['FinancDepth','RealGdpPerCapita','MerchanTrade','LnInflation']])
fitted = model.fit(q=0.1)
print(fitted.summary())

                         QuantReg Regression Results                          
Dep. Variable:                GdpRate   Pseudo R-squared:               -10.70
Model:                       QuantReg   Bandwidth:                     0.01967
Method:                 Least Squares   Sparsity:                       0.5304
Date:                Sun, 04 Dec 2016   No. Observations:                  101
Time:                        20:35:44   Df Residuals:                       97
                                        Df Model:                            4
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
FinancDepth          0.0002      0.001      0.447      0.656      -0.001       0.001
RealGdpPerCapita -2.926e-06   1.71e-06     -1.711      0.090   -6.32e-06    4.68e-07
MerchanTrade         0.0007      0.000      1.542      0.126      -0.000       0.002
LnInflation         -0

## Q=0.25

In [19]:
model = QuantReg(finalSample['GdpRate'], finalSample[['FinancDepth','RealGdpPerCapita','MerchanTrade','LnInflation']])
fitted = model.fit(q=0.25)
print(fitted.summary())

                         QuantReg Regression Results                          
Dep. Variable:                GdpRate   Pseudo R-squared:               -10.90
Model:                       QuantReg   Bandwidth:                     0.01914
Method:                 Least Squares   Sparsity:                       0.4333
Date:                Sun, 04 Dec 2016   No. Observations:                  101
Time:                        20:35:44   Df Residuals:                       97
                                        Df Model:                            4
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
FinancDepth         -0.0004      0.000     -0.862      0.391      -0.001       0.001
RealGdpPerCapita -2.602e-06   1.57e-06     -1.662      0.100   -5.71e-06    5.05e-07
MerchanTrade         0.0005      0.001      0.935      0.352      -0.001       0.002
LnInflation         -0

## Q=0.5

In [20]:
model = QuantReg(finalSample['GdpRate'], finalSample[['FinancDepth','RealGdpPerCapita','MerchanTrade','LnInflation']])
fitted = model.fit(q=0.5)
print(fitted.summary())

                         QuantReg Regression Results                          
Dep. Variable:                GdpRate   Pseudo R-squared:               -10.56
Model:                       QuantReg   Bandwidth:                     0.02139
Method:                 Least Squares   Sparsity:                       0.3510
Date:                Sun, 04 Dec 2016   No. Observations:                  101
Time:                        20:35:44   Df Residuals:                       97
                                        Df Model:                            4
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
FinancDepth         -0.0003      0.000     -0.837      0.405      -0.001       0.000
RealGdpPerCapita -9.584e-06   1.63e-06     -5.869      0.000   -1.28e-05   -6.34e-06
MerchanTrade         0.0013      0.000      3.271      0.001       0.001       0.002
LnInflation         -0

## Q=0.75

In [21]:
model = QuantReg(finalSample['GdpRate'], finalSample[['FinancDepth','RealGdpPerCapita','MerchanTrade','LnInflation']])
fitted = model.fit(q=0.75)
print(fitted.summary())

                         QuantReg Regression Results                          
Dep. Variable:                GdpRate   Pseudo R-squared:               -9.983
Model:                       QuantReg   Bandwidth:                     0.01914
Method:                 Least Squares   Sparsity:                       0.3384
Date:                Sun, 04 Dec 2016   No. Observations:                  101
Time:                        20:35:44   Df Residuals:                       97
                                        Df Model:                            4
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
FinancDepth         -0.0013      0.000     -3.437      0.001      -0.002      -0.001
RealGdpPerCapita -2.036e-06   1.84e-06     -1.107      0.271   -5.69e-06    1.62e-06
MerchanTrade         0.0033      0.000     10.725      0.000       0.003       0.004
LnInflation         -0

## Q=0.9

In [22]:
model = QuantReg(finalSample['GdpRate'], finalSample[['FinancDepth','RealGdpPerCapita','MerchanTrade','LnInflation']])
fitted = model.fit(q=0.9)
print(fitted.summary())

                         QuantReg Regression Results                          
Dep. Variable:                GdpRate   Pseudo R-squared:               -9.486
Model:                       QuantReg   Bandwidth:                     0.01967
Method:                 Least Squares   Sparsity:                       0.4381
Date:                Sun, 04 Dec 2016   No. Observations:                  101
Time:                        20:35:44   Df Residuals:                       97
                                        Df Model:                            4
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
FinancDepth         -0.0012      0.000     -2.533      0.013      -0.002      -0.000
RealGdpPerCapita -3.352e-06   2.61e-06     -1.285      0.202   -8.53e-06    1.83e-06
MerchanTrade         0.0059      0.000     16.824      0.000       0.005       0.007
LnInflation         -0